Integrating the SceneGraph notebooks with the Toybot notebooks

This includes code to run the bot as script.

SceneGraph
==

The scenegraph stores operations as nodes.

These include that change state (fill, stroke etc) and operations that result in elements being drawn (path)

In [13]:
class SceneGraph:
    def __init__(self):
        self.nodes = []
        
    def add_node(self, node):
        self.nodes.append(node)
    
    def __iter__(self):
        """
        Iterator that yields every node in the graph
        """
        for node in self.nodes:
            yield node

## Nodes

### Path

We'll start with the simplest shape -- a path made of straight lines -- and later implement new ones once we agree on a clear structure for the node/graph architecture.

It should only need:

* the minimum number of vars to store values
* methods to edit the path
* a method to dump as JSON

We will probably want a `Node` base class, but let's start simple.

In [14]:
class Path:
    def __init__(self):
        self.coords = []
    
    def add_point(self, x, y):
        self.coords.append((x, y))
    
    def as_dict(self):
        return {'type': 'path', 'coords': self.coords}
    
    def as_json(self):
        import json
        return json.dumps(self.as_dict())

Let's try it now:

In [15]:
p = Path()
p.add_point(10, 10)
p.add_point(10, 100)
p.add_point(100, 100)
p.add_point(100, 10)

print(p.as_json())

{"type": "path", "coords": [[10, 10], [10, 100], [100, 100], [100, 10]]}


Now it's time to create an instance of our scene graph and add this path to it, so we can render it later.

In [16]:
graph = SceneGraph()
graph.add_node(p)

ConsoleRenderer
==

The ConsoleRenderer outputs every node of scenegraph to the console

In [17]:
class ConsoleRenderer:
    def render(self, graph):
        for node in graph:
            print(node.as_json())

Now render our graph:

In [18]:
renderer = ConsoleRenderer()
renderer.render(graph)

{"type": "path", "coords": [[10, 10], [10, 100], [100, 100], [100, 10]]}


We could also set `__str__` on the Node object so it returns a prettier output, but while ugly, JSON is handy if we want to pass data around.

Grammar
==

The ToyBot class provides the user-facing API for drawing and setting up colours.

In [19]:
class ToyBot:
    def __init__(self, graph):
        self.graph = graph
    
    def rect(self, x, y, width, height, fill=None, stroke=None):
        coords = [(x, y), (x, y+height), (x+width, y+height), (x+width, y)]
        p = Path()
        # Let's just set coords directly instead of calling add_node repeatedly
        p.coords = coords        
        self.graph.add_node(p)

# Scripting

## Setup the scripting namespace

This function adds all the user-facing API of a bot into a namespace to enable scripting.

In [20]:
import inspect

def create_scripting_namespace(bot):
    namespace = {}
    for name, method in inspect.getmembers(bot, predicate=inspect.ismethod):
        if name.startswith('__'):
            continue
        namespace[name] = method
    return namespace

**Note:** This is only gathering functions, not variables.

## Running a script

The run function accepts a bot script, does the setup needed to render a bot and then renders.

In [21]:
def run(source):
    graph = SceneGraph()
    renderer = ConsoleRenderer()

    bot = ToyBot(graph)
    namespace = create_scripting_namespace(bot)
    
    exec(source, namespace)

    renderer.render(graph)

## Putting it all together - using the scripting interface

The scripting interface provides the simplest way of using ToyBot.

Pass the code to render to run function.

In [22]:
code = """
rect(0, 0, 100, 100)
rect(25, 30, 50, 70)
"""

run(code)

{"type": "path", "coords": [[0, 0], [0, 100], [100, 100], [100, 0]]}
{"type": "path", "coords": [[25, 30], [25, 100], [75, 100], [75, 30]]}
